In [2]:
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  

import tools.Sample_Tools as smpl
import tools.Pretreat_Tools as pretreat
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)

from base.JuUnits import parallal_task,task_chunk_split
from base.JuUnits import excute_for_multidates

import ind.Ind_MACD_TRE as macd


%load_ext autoreload
%autoreload 2
%aimport ind.Ind_MACD_TRE



In [5]:
#读取已经缓存的复权数据，并预处理
tail = True
origin_start = None
if tail:
    df_tail = load_cache('all_tail_qfq',cache_type=CACHE_TYPE.STOCK).sort_index()
    df_train = load_cache('all_train_qfq',cache_type=CACHE_TYPE.STOCK).sort_index()
    origin_start = df_tail.index.get_level_values(0).min()
    year_data_reserve = origin_start.year - 2
    stock_df = pd.concat([df_train.loc[df_train.index.get_level_values(0) >= str(year_data_reserve)],df_tail],axis=0)
else:
    stock_df = load_cache('all_train_qfq',cache_type=CACHE_TYPE.STOCK).sort_index()
smpl.optimize_data_type(stock_df)
df_tail = None
df_train = None


#############测试#######################
# ['000001','000008','600600','000729']
# stock_df = stock_df.loc[(slice(None),stock_df.index.get_level_values(1).unique().to_list()[0:6]),:]

stock_codes = stock_df.index.get_level_values(1).unique().to_list()
np.random.shuffle(stock_codes)


def generate_macd_factors(data_df):
    import ind.Ind_MACD_TRE as macd
    import numpy as np

    df = data_df.close
    res = macd.MACD_JCSC(df)

    return res

worker=6
task = task_chunk_split(stock_codes, worker)
task_dfs=[]
for codes in task:
    task_dfs.append(stock_df.loc[(slice(None),codes),])

results = parallal_task(worker, generate_macd_factors, task_dfs)
results = pd.concat(results).sort_index()
results.index.names=['date','code']



final = results[['MACD','MACD_CROSS','DEVIATE']]
if tail:
    save_cache('macd_tail', final.loc[origin_start:], cache_type=CACHE_TYPE.FACTOR)
else:
    save_cache('macd_train', final, cache_type=CACHE_TYPE.FACTOR)

Now in the main code. Process name is: base.JuUnits
base.JuUnits, subpid:18716  pid:9696


  0%|          | 0/6 [00:00<?, ?it/s]

In [6]:
# load_cache('base10_train',cache_type=CACHE_TYPE.FACTOR).dropna()
# stock_df.loc[(slice(None),['000001','000002']),:][['close']]
stock_df.loc[(slice(None),['000001']),:].shape

(1218, 12)

In [7]:
final.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3009425 entries, (Timestamp('2016-01-04 00:00:00'), '000001') to (Timestamp('2020-12-31 00:00:00'), '605500')
Data columns (total 3 columns):
 #   Column   Dtype  
---  ------   -----  
 0   MACD     float64
 1   CROSS    int64  
 2   DEVIATE  float64
dtypes: float64(2), int64(1)
memory usage: 80.5+ MB


In [4]:
x = load_cache('macd_train', cache_type=CACHE_TYPE.FACTOR)
x

MACD  CROSS  DEVIATE
date       code                            
2016-01-04 000001       NaN      0      NaN
           000004       NaN      0      NaN
           000006       NaN      0      NaN
           000008       NaN      0      NaN
           000009       NaN      0      NaN
...                     ...    ...      ...
2020-12-31 605376       NaN      0      NaN
           605377       NaN      0      NaN
           605388 -0.389375      0      0.0
           605399 -0.179166      0      9.0
           605500       NaN      0      NaN

[3009425 rows x 3 columns]